In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q langchain langchain_community langchain_openai sentence_transformers chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
import json
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=device)

<ipython-input-5-f5eeaa36f5a7>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def split_dialogue_by_empty_lines(conversation):
    dialogue_units = conversation.strip().split('\n\n')
    return dialogue_units

In [ ]:
file_path = '/content/drive/Shareddrives/298B/DATA298B/data/Sherlock_Holmes_QA.txt'

data = []
with open(file_path, 'r') as file:
    for line in file:
        entry = json.loads(line)
        question = entry.get("question")
        generated_answer = entry.get("generated", [""])[0]
        data.append({"Prompt": question, "Quote": generated_answer})

df = pd.DataFrame(data)

split_dialogues = []

for idx, row in df.iterrows():
    prompt = row['Prompt']
    quote = row['Quote']
    dialogue = f"Prompt: {prompt}\nQuote: {quote}"
    split_dialogues.append(dialogue)

documents = [Document(page_content=dialogue) for dialogue in split_dialogues]
documents

[Document(metadata={}, page_content="Prompt: Sherlock Holmes, how do you develop your exceptional combat skills?\nQuote: My combat skills were developed through years of rigorous training, both physical and mental. I studied various martial arts, learned how to use different weapons, and honed my reflexes and agility. However, my intellectual prowess is just as important in combat situations, allowing me to analyze my opponents' weaknesses and predict their movements before they even make them."),
 Document(metadata={}, page_content='Prompt: Sherlock Holmes, can you explain how you solve complex crimes using your deductive reasoning?\nQuote: Deductive reasoning is a method of logical thinking that involves drawing conclusions based on available evidence. I carefully observe and analyze every aspect of a crime, from physical evidence to behavior patterns, and use logical steps to arrive at a conclusion. I also use my vast knowledge of science, history, and literature to draw comparisons

In [ ]:
SH_db = Chroma.from_documents(documents=documents, embedding=embeddings,persist_directory="/content/drive/Shareddrives/298B/DATA298B/db/SH")
SH_db.persist()

<ipython-input-10-40b8632609a1>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  SH_db.persist()


In [ ]:
db = Chroma(persist_directory="/content/drive/Shareddrives/298B/DATA298B/db/SH", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={'k': 10})

<ipython-input-11-a3346de048f2>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="/content/drive/Shareddrives/298B/DATA298B/db/SH", embedding_function=embeddings)
